In [1]:
import trimesh
import numpy as np
import pyigl  as igl 


def union_meshes(m1, m2):
    '''
    Union two meshes in trimesh. 
    only union the vertices, faces, and vertex colors.
    input:
        m1: the first mesh
        m2: the second mesh
    output:
        m3: the merged mesh.    
    '''
    
    # get the vertices, faces, and vertex color.
    v1 = np.array(m1.vertices)
    v2 = np.array(m2.vertices)
    f1 = np.array(m1.faces)
    f2 = np.array(m2.faces)
    c1 = np.array(m1.visual.vertex_colors)
    c2 = np.array(m2.visual.vertex_colors)

    # merge the meshes
    v = np.concatenate((v1, v2), axis=0)
    f = np.concatenate((f1, f2 + np.amax(f1) + 1), axis=0)
    c = np.concatenate((c1, c2), axis=0)
    
    # create a new mesh
    m3 = trimesh.Trimesh(vertices=v, faces=f, process=False)
    m3.visual.vertex_colors = c
    
    return m3

def l2_normalization(n):
    '''
    Normalize a vector.
    input:
        n
    output:
        n
    '''

    # normaliza n
    n = n / np.sqrt(np.sum(n * n, axis=-1, keepdims=True))

    return n

def get_rotation_angle(n):
    '''
    input:
        n: the normal direction (3,)
    output:
        Ry: the rotation matrix along y axis.
        Rz: the rotation matrix along z axis.
    '''

    # get the x, y, z coordinate
    x = n[0]
    y = n[1]
    z = n[2]

    # get angles
    if y >= 0:
        alpha = np.arccos(x / np.sqrt(x * x +y * y))
    else:
        alpha = 2 * np.pi - np.arccos(x / np.sqrt(x * x +y * y))

    beta = np.arctan(z / np.sqrt(x * x + y * y))

    return alpha, beta

def get_rotation_matrix_from_angle(alpha, beta):
    '''
    input:
        alpha:
        beta:
    output:
        Ry:
        Rz:
    '''

    # compute the rotation matrix
    Ry = np.array([[np.cos(beta), 0, -np.sin(beta)],[0, 1, 0], [np.sin(beta), 0, np.cos(beta)]])
    Rz = np.array([[np.cos(alpha), -np.sin(alpha), 0], [np.sin(alpha), np.cos(alpha), 0], [0, 0, 1]])

    return Ry, Rz

def get_rotation_matrix(n):
    '''
    input:
        n: the normal direction
    output:
        Ry: the rotation matrix along y axis.
        Rz: the rotation matrix along z axis.
    '''

    # normaliza n
    n = l2_normalization(n)

    # compute the angle
    alpha, beta = get_rotation_angle(n)

    # compute the rotation matrix
    Ry = np.array([[np.cos(beta), 0, -np.sin(beta)],[0, 1, 0], [np.sin(beta), 0, np.cos(beta)]])
    Rz = np.array([[np.cos(alpha), -np.sin(alpha), 0], [np.sin(alpha), np.cos(alpha), 0], [0, 0, 1]])

    return Ry, Rz

def srt_axis(Ry, Rz, t, s, axis):
    '''
    input: 
        Ry
        Rz
        t:
        s
    output:
        m: the rotated x axis
    '''

    # load axis
    if axis == 'x':
        m = trimesh.load_mesh('x_axis_001.obj', process=False)
    elif axis == 'y':
        m = trimesh.load_mesh('y_axis_001.obj', process=False)
    else:
        m = trimesh.load_mesh('z_axis_001.obj', process=False)
    m.vertices = (Rz @ Ry @ (m.vertices * s).T).T + t

    return m

def get_curvature_direction(file_name):
    '''
    input: 
        file_name
    output:
        pd1:
        pd2:
    '''
    
    # load the mesh
    V = igl.eigen.MatrixXd()
    F = igl.eigen.MatrixXi()
    igl.read_triangle_mesh(file_name, V, F)

    # Compute curvature directions via quadric fitting
    PD1 = igl.eigen.MatrixXd()
    PD2 = igl.eigen.MatrixXd()

    PV1 = igl.eigen.MatrixXd()
    PV2 = igl.eigen.MatrixXd()

    igl.principal_curvature(V, F, PD1, PD2, PV1, PV2)
    
    # convert to numpy array
    pd1 = np.array(PD1)
    pd2 = np.array(PD2)

    return pd1, pd2

def clean_max_curvature_direction(normal, pd1):
    '''
    Clean the maximum curvature direction given the normal direction.
    input:
        normal: the normal direction (n, 3)
        pd1: the pricinpal direction (n, 3)
    output:
        pd1: the cleaned pricinpal direction (n, 3)
    '''
    
    # project the maximum direction on to the normal
    projection = np.sum(normal * pd1, axis=-1, keepdims=True) # (n, 1)
    
    # get the new pd1
    pd1 = pd1 - projection * normal
    
    # l2 normalize the pd1
    pd1 = l2_normalization(pd1)
    
    return pd1

def get_LRF(file_name):
    '''
    Compute local reference frame given the file name.
    input:
        file_name: the file name of the obj file
    output:
        lrf: the local reference frame (n, 3, 3)    
    '''
    
    # compute the x, y, z
    m = trimesh.load_mesh(file_name, process=False)
    n = m.vertex_normals
    pd1, pd2 = get_curvature_direction(file_name)
    pd1 = clean_max_curvature_direction(n, pd1)
    pd2 = np.cross(n, pd1)
    
    # concatenate x, y, z together
    lrf = np.concatenate(
        (np.expand_dims(pd1, -1), np.expand_dims(pd2, -1), np.expand_dims(n, -1)), 
        axis=-1
    )
    
    return lrf

def visualize_lrf(lrf, mesh):
    '''
    input:
        lrf: the local reference frame (n, 3, 3)
        mesh: the Trimesh object
    output:
        mesh: the mesh to be save
    '''
    
    # get the vertices
    v = np.array(mesh.vertices)
    
    # add the axis
    for i in range(v.shape[0]):

        # get the x axis
        nx = lrf[i, :, 0]
        t = v[i, :]
        Ry, Rz = get_rotation_matrix(nx)
        mx = srt_axis(Ry, Rz, t, 0.006, 'x')
        mesh = union_meshes(mesh, mx)

        # get the y axis
        ny = lrf[i, :, 1]
        t = v[i, :]
        Ry, Rz = get_rotation_matrix(ny)
        my = srt_axis(Ry, Rz, t, 0.006, 'y')
        mesh = union_meshes(mesh, my)

        # get the y axis
        nz = lrf[i, :,2]
        t = v[i, :]
        Ry, Rz = get_rotation_matrix(nz)
        mz = srt_axis(Ry, Rz, t, 0.006, 'z')
        mesh = union_meshes(mesh, mz)

    return mesh


In [2]:
import os
import glob

# os.mkdir('../ver')
# os.mkdir('../normal')
# os.mkdir('../lrf')

mesh_file_names = glob.glob('../mesh/*.off')

for mesh_file_name in mesh_file_names:

    # save name template
    name_tmp = mesh_file_name[8:-4]

    # save local reference frame
    lrf = get_LRF(mesh_file_name)
    np.save('../lrf/' + name_tmp, lrf)

    # load the mesh
    mesh = trimesh.load_mesh(mesh_file_name, process=False)

    # save vertices for adobe the vertices should be diveded by 100
    ver = mesh.vertices
    np.save('../ver/' + name_tmp, ver)

    # save normal
    normal = mesh.vertex_normals
    np.save('../normal/' + name_tmp, normal)
